<p style='text-align: justify;'>After the $n_k(m)$ solved, the potential field can be differentiated to obtain the field on different components. This code takes an input set of X, Y and Z magnetic field components and from them calculates a spherical cap harmonic model of the magnetic field within the region of a half degree radius cap around a point specified by the user. The declination, inclination and intensity of the model field are then plotted on a map. The spherical cap harmonic calculation is much more complicated than spherical harmonic, with each legendre polynomial being a summation, and n being non-integer.</p>

In [120]:
import pickle
import pandas as pd
import numpy as np
import mathimport osimport os.path

<p style='text-align: justify;'>Define function to calculate distance on a sphere.</p>

In [121]:
def distance_on_unit_sphere(lat1, long1, lat2, long2):
    # Convert latitude and longitude to spherical coordinates in radians.
    degrees_to_radians = math.pi/180.0
    
    # phi = 90 - latitude
    phi1 = (90.0 - lat1)*degrees_to_radians
    phi2 = (90.0 - lat2)*degrees_to_radians
    
    # theta = longitude
    theta1 = long1*degrees_to_radians
    theta2 = long2*degrees_to_radians
    
    # Compute spherical distance from spherical coordinates.
    cos = (math.sin(phi1)*math.sin(phi2)*math.cos(theta1 - theta2) + math.cos(phi1)*math.cos(phi2))
    arc = math.acos( cos )
    
    return arc

<p style='text-align: justify;'>The geomagnetic measurements data positions and components on the Earth's surface are usually given in a geographic coordinate system. Therefore, it is necessary to transform both coordinate locations and the vector-pointing directions to the spherical cap coordinate system to simplify calculations. From the spherical cap coordinate system, the transformed longitude and colatitude for any data site can be found by considering a triangle on the Earth’s surface with the centre of the spherical cap, the North geographic pole, and the data points as shown in following figure.</p>

<img src="img/transformation.png" width="400"/>

<p style='text-align: justify;'>Consider an arbitrary point $P$ in the geographic coordinate system which has coordinate $(\varphi_P,\theta_P)$ and the spherical cap pole with coordinate $(\varphi_C,\theta_C)$. The meridian of these points $(\theta_C)$ and $(\theta_P)$ pass through the centre of the cap and data sites, and simply become the sides of the spherical triangle with longitude difference between these two sides become the angle $(\varphi_C-\varphi_P)$. Using the spherical version of the cosine rule, the transformed colatitude $(\theta_T)$ of the data point, i.e. relative to the centre of the cap, can be determined as follows \begin{align}\theta_T=\cos^{-1⁡}(\cos\,⁡\theta_C\;\cos\,⁡\theta_P+\sin⁡\,\theta_C\;\sin\,\theta_P\;\cos\,⁡(\varphi_C-\varphi_P))\end{align}</p>

In [122]:
def sph_cosine_side(a, b, C):
    # Input and output in radian
    c = np.arccos(np.cos(a)*np.cos(b)+np.sin(a)*np.sin(b)*np.cos(C))
    return c

<p style='text-align: justify;'>While the transformed longitude $(\varphi_T)$ can be determined using the spherical version of the cosine rule as follows.</p>
\begin{align}\varphi_T=\cos^{-1} \left (\frac{\cos\,\theta_P-\cos⁡\,\theta_T\;\cos⁡\,\theta_C}{\sin⁡\,\theta_T\;\sin\,\theta_C} \right)\end{align}
<p></p>
<p style='text-align: justify;'>Similarly, the geomagnetic data at point $P$ also need to be transformed into the cap coordinate system. The rotation angle $\gamma$ needs to be determined for every data point using the spherical version of the cosine rule.</p>
\begin{align}\gamma=\cos^{-1} \left (\frac{\cos\,\theta_C-\cos⁡\,\theta_T\;\cos⁡\,\theta_P}{\sin⁡\,\theta_T\;\sin\,\theta_P} \right)\end{align}

In [123]:
def sph_cosine_angle(a, b, c):
    C = np.arccos((np.cos(c)-np.cos(a)*np.cos(b))/(np.sin(a)*np.sin(b)))
    return C

<p style='text-align: justify;'>The Schmidt normalised Associated Legendre Polynomials in the solution above can be determined using</p>

\begin{align}P_n^m (\cos\,\theta)=\sum_{k=0}^K A_k (m,n) \; {\sin^{2k} \, (\theta/2)} \end{align}

In [124]:
def assoc_Leg_func(n, A, thetaT):
    P = np.zeros(len(n))
    for i in range(len(n)):
        for j in range(np.size(A,1)):
            P[i] = P[i] + A[i,j]*(np.sin(thetaT/2)**(2*j))
    return P

<p style='text-align: justify;'>while the derivation of Schmidt normalised Associated Legendre Polynomials is</p>

\begin{align}\frac{\partial P_n^m (\cos\,\theta)}{\partial \theta} = \frac{\sin\,\theta}{2} \sum_{k=1}^K kA_k (m,n)\;\sin^{2(k-1)} ⁡(\theta/2) + \cos\,\theta\; \left [\frac{m}{\sin\,\theta} \; P_n^m (\cos⁡\,\theta) \right]\end{align}

In [125]:
def deriv_assoc_Leg_func(n, m, A, thetaT, P):
    sigma = np.zeros(len(n))
    dP = np.zeros(len(n))
    for i in range(len(n)):
        for j in range(1,np.size(A,1)):
            sigma[i] = sigma[i] + j*A[i,j]*(np.sin(thetaT/2)**(2*(j-1)))
        dP[i] = np.sin(thetaT)/2*sigma[i] + np.cos(thetaT)*(m[i]/np.sin(thetaT)*P[i])
    return dP

<p style='text-align: justify;'>The eastern component of geomagnetic field can be found as follows.</p>

\begin{align}B_x=\frac{1}{r} \frac{\partial U(r,\theta,\varphi)} {\partial \theta}\end{align}

\begin{align}=\sum_{m=0}^\infty \, \sum_{k=m}^\infty \left (\frac{a}{r} \right)^{n_k(m)+2} \cdot \frac{\partial P_{n_k(m)}^m \; (\cos⁡ \,\theta_T)}{\partial \theta_T} \; \{g_k^m  \cos⁡ \,m\varphi_T + h_k^m  \sin⁡ \, m\varphi_T\}\end{align}

In [126]:
def Bx(m, dP, phiT, thetaT):
    X_coeff=[]
    for r in range(len(m)):
        if m[r]==0: # m=0, we only compute g
            X_coeff = X_coeff + [(np.sin(thetaT)**(m[r]))*dP[r]*np.cos(m[r]*phiT)] # g
        else: # Otherwise, we need to compute g and h
            X_coeff = X_coeff + [(np.sin(thetaT)**(m[r]))*dP[r]*np.cos(m[r]*phiT)] # g
            X_coeff = X_coeff + [(np.sin(thetaT)**(m[r]))*dP[r]*np.sin(m[r]*phiT)] # h
    return X_coeff

<p style='text-align: justify;'>The northern component of geomagnetic field can be found as follows.</p>

\begin{align}B_y = - \frac{1}{r \sin\,\theta} \; \frac{\partial U(r,\theta,\varphi)} {\partial \varphi}\end{align}

\begin{align}=\sum_{m=0}^\infty \sum_{k=m}^\infty m \left(\frac{a}{r} \right)^{n_k (m)+2} \cdot \frac{P_{n_k(m)}^m \; (\cos⁡\,\theta_T)} {\sin⁡\,\theta_T} \; \{g_k^m \sin⁡ \,m\varphi_T - h_k^m \cos⁡ \,m\varphi_T\}\end{align}

In [127]:
def By(n, m, P, phiT, thetaT):
    Y_coeff=[]
    for r in range(len(m)):
        if m[r]==0: # m=0, we only compute g
            Y_coeff = Y_coeff + [(np.sin(thetaT)**(m[r]))*m[r]/np.sin(thetaT)*P[r]*np.sin(m[r]*phiT)] # g
        else: # Otherwise, we need to compute g and h
            Y_coeff = Y_coeff + [(np.sin(thetaT)**(m[r]))*m[r]/np.sin(thetaT)*P[r]*np.sin(m[r]*phiT)] # g
            Y_coeff = Y_coeff + [(np.sin(thetaT)**(m[r]))*(-m[r])/np.sin(thetaT)*P[r]*np.cos(m[r]*phiT)] # g
    return Y_coeff

<p style='text-align: justify;'>The vertical component of geomagnetic field can be found as follows.</p>

\begin{align}B_z=\frac{\partial U(r,\theta,\varphi)}{\partial r}\end{align}

\begin{align}=-\sum_{m=0}^\infty \sum_{k=m}^\infty \; (n_k(m)+1) \; \left (\frac{a}{r} \right)^{n_k(m)+2} \cdot P_{n_k(m)}^m \; (\cos⁡ \,\theta_T) \; \{g_k^m  \cos⁡\,m\varphi_T + h_k^m \sin⁡\,m\varphi_T\}\end{align}

In [128]:
def Bz(n, m, P, phiT, thetaT):
    Z_coeff = []
    for r in range(len(m)):
        if m[r]==0: # m=0, we only compute g
            Z_coeff = Z_coeff + [(np.sin(thetaT)**m[r])*(-(n[r]+1)*P[r]*np.cos(m[r]*phiT))] # g
        else: # Otherwise, we need to compute g and h
            Z_coeff = Z_coeff + [(np.sin(thetaT)**m[r])*(-(n[r]+1)*P[r]*np.cos(m[r]*phiT))] # g
            Z_coeff = Z_coeff + [(np.sin(thetaT)**m[r])*(-(n[r]+1)*P[r]*np.sin(m[r]*phiT))] # h     
    return Z_coeff

<p style='text-align: justify;'>The model parameter can be determined here</p>

In [129]:
# Order of the model
mmax = 7
# Spherical cap parameter
Latcentre = -3
Longcentre = 122
theta0 = 30
year = 2015

<p style='text-align: justify;'>The data used in this study are the X, Y, and Z components compiled from 68 geomagnetic repeat stations in Indonesia, covering the period 1985 - 2015 from BMKG (Badan Meteorologi Klimatologi dan Geofisika). However, two locations are not included in the calculation and will be used later to validate the model. These validation observatories are Bandar Lampung (105.175, -5.240) and Tual (132.735, -5.662). The repeat station data are combined with definitive data from five BMKG’s geomagnetic observatories from 1985 - 2019. To complete the dataset, the definitive data from a similar period from 13 INTERMAGNET observatories are used. In spherical cap harmonic modelling it is essential to have data around the edges of the chosen spherical cap. Because of the irregularities in the data distribution in the research area, the synthetic cartesian X, Y, and Z components at sea level at 17 fixed points are calculated from IGRF-13 at five-year intervals from 1985 – 2015. The IGRF-13 is also used to remove the main field before applying the techniques to calculate the regional field.</p>

<img src="img/data_points.png" width="400"/>

<p style='text-align: justify;'>In this research, the coordinate of the spherical cap pole is at 122$^{\circ}$E and 3$^{\circ}$S, and the spherical cap half-angle is 30$^{\circ}$. This covers the Indonesia region, as shown in the following figure. According to Torta, Gaya-Piqué, & De Santis (2006), the use of a large spherical cap is necessary to obtain the main field and its secular variation; otherwise, the results are unrealistic. Research by Qamili et al. (2010) used a half-angle of 16$^{\circ}$, although their data mostly concentrated in the central 4$^{\circ}$ half-angle. Torta, García, Curto, & De Santis (1992) and Torta & De Santis (1996) explain that the RMS residual of the SCHA model decreases as the cap half-angle increase. However, Düzgit & Malin (2000) obtained different results and reported that the quality of the fit of the SCHA model is relatively insensitive to the cap radius. Nevertheless, the cap half-angle chosen in this research is the most optimal as it is cover the Indonesia region, and there are several geomagnetic observatories located near the boundary of the cap.</p>

In [130]:
# Load sample data
with open(os.path.join(os.path.expanduser('~'), 'data/2015.pkl'), 'rb') as f:
    df = pickle.load(f)
with open(os.path.join(os.path.expanduser('~'), 'data/Grid_IGRF.pkl'), 'rb') as f:
    data_grid_IGRF = pickle.load(f)
# Load A and n from nk_solver.py
with open(os.path.join(os.path.expanduser('~'), 'data/output_'+str(mmax)+'_'+str(theta0)+'.pkl'), 'rb') as f:
    n,A,Kmn = pickle.load(f)

# Gives names to different data columns (assuming order as above)
Lat = df['Lat']
Long = df['Lon']
Xobs = df['X Obs']
Yobs = df['Y Obs']
Zobs = df['Z Obs']
XIGRF = df['X IGRF']
YIGRF = df['Y IGRF']
ZIGRF = df['Z IGRF']

# Remove main field
Xres = Xobs-XIGRF
Yres = Yobs-YIGRF
Zres = Zobs-ZIGRF

# Number of data
num_data = len(Lat)

<p style='text-align: justify;'>Convert coordinate into radians</p>

In [131]:
m = []
for a in range(mmax + 1):     # We need to add 1 because python list start at 0
    b = list(range(a + 1))
    m = m + b

# Colatitude of the center of the cap
colatradc = (90-Latcentre)*np.pi/180
# Longitude of the center of the cap
longradc = Longcentre*np.pi/180

# spherical harmonic analysis works with colatitude, and in radians, so convert original data
colat = 90-Lat
# Colatitude of the data
colatrad = colat*np.pi/180
# Longitude of the data
longrad = Long*np.pi/180
# number of data sites
num = len(colatrad)

<p style='text-align: justify;'>Firstly need to rotate given X, Y and Z values so that they are in the coordinate system of the cap</p>

In [132]:
colatcap = sph_cosine_side(colatradc, colatrad, longradc-longrad)
longcap = np.zeros(num)
gamma = np.zeros(num)

# longitude in cap
for i in range(num):
    if Long[i]==Longcentre and Lat[i]>=Latcentre: # all points above centre on meridian
        longcap[i] = 0
    elif Long[i]==Longcentre and Lat[i]<Latcentre: # all points below centre on meridian
        longcap[i] = np.pi
    else:
        longcap[i] = sph_cosine_angle(colatradc, colatcap[i], colatrad[i])
    
    # also need to rotate the X and Y values by angle gamma, such that:
    if Long[i]==Longcentre and Lat[i]<=Latcentre:        # points below centre, on meridian
        gamma[i] = 0.0
    elif Long[i]==Longcentre and Lat[i]>Latcentre: # points above centre, on meridian
        gamma[i] = np.pi
    else:
        gamma[i] = sph_cosine_angle(colatrad[i], colatcap[i], colatradc)
        
    # place the longitudes and gammas in right quadrant
    if longrad[i]>longradc:
        longcap[i] = 2*np.pi-longcap[i]
        gamma[i] = 2*np.pi-gamma[i]
        

<p style='text-align: justify;'>Rotate the geomagnetic components. The transformed geomagnetic data for the X and Y components can be determined as<\p>

\begin{align} B_X^\prime=B_X \cos\,\gamma + B_Y \sin\,\gamma \end{align}
    
\begin{align} B_Y^\prime=-B_X \sin\,\gamma + B_Y \cos\,\gamma \end{align}
    
\begin{align} B_Z^\prime=B_Z \end{align}
    
<p style='text-align: justify;'>The downward component ($B_Z$) is unaffected by the coordinate transformation</p>

In [133]:
Xcap = Xres*np.cos(gamma) + Yres*np.sin(gamma)
Ycap = -Xres*np.sin(gamma) + Yres*np.cos(gamma)
Zcap = Zres

<p style='text-align: justify;'>Calculates the Gauss coefficients</p>

In [134]:
# Each coefficient consists of two parts, each with a summation over the A(m,n) values as given in the matrix above.
field = np.zeros(3*num)
coeff = list(np.zeros((3*num)))

for i in range(num):
    # Associate Legendre function
    P = assoc_Leg_func(n, A, colatcap[i])
    # Derived associate Legendre function
    dP = deriv_assoc_Leg_func(n, m, A, colatcap[i], P)
    
    # X coefficients are 1/r dU/d(colatcap)
    coeff[i] = Bx(m, dP, longcap[i], colatcap[i])
    field[i] = Xcap[i]
    
    # Y coefficients are -1/rsin(colatcap) dU/d(longcap)
    coeff[num+i] = By(n, m, P, longcap[i], colatcap[i])
    field[num+i] = Ycap[i]
    
    # Z coefficients are du/dr
    coeff[2*num+i] = Bz(n, m, P, longcap[i], colatcap[i])
    field[2*num+i] = Zcap[i]

<p style='text-align: justify;'>The gauss coefficients are found by a least squares method for the over determined set of equations.</p>

In [135]:
gauss = np.linalg.lstsq(coeff,field,rcond=None)[0]

<p style='text-align: justify;'>Calculates the statistics of the model (Mean Absolute Error (MAE) and Root Mean Square Deviation (RMSD))</p>

In [136]:
# Calculate the Mean Absolute Error (MAE) or mean difference
# Need to calculate the modeled field first (field=model*gauss)
Xmodcap = np.dot(coeff[0:num_data],gauss)              # X model field in cap coordinate
Ymodcap = np.dot(coeff[num:num+num_data],gauss)      # Y model field in cap coordinate
Zmodcap = np.dot(coeff[2*num:2*num+num_data],gauss)  # Z model field in cap coordinate

Xmod = Xmodcap*np.cos(gamma[0:num_data])-Ymodcap*np.sin(gamma[0:num_data])   # X model field in geographic coordinate
Ymod = Xmodcap*np.sin(gamma[0:num_data])+Ymodcap*np.cos(gamma[0:num_data])  # Y model field in geographic coordinate
Zmod = Zmodcap                           # Z model field in geographic coordinate

XMAE = np.mean(abs(Xmod-Xres[0:num_data]))
YMAE = np.mean(abs(Ymod-Yres[0:num_data]))
ZMAE = np.mean(abs(Zmod-Zres[0:num_data]))

# Calculate the Root Mean Square Deviation (RMSD) or RMS difference
XRMSD = np.sqrt(np.mean((Xmod-Xres[0:num_data])**2))
YRMSD = np.sqrt(np.mean((Ymod-Yres[0:num_data])**2))
ZRMSD = np.sqrt(np.mean((Zmod-Zres[0:num_data])**2))

<p style='text-align: justify;'>Calculates the SCHA model over the whole cap on a grid</p>

In [137]:
# As with the spherical harmonic code, want to calculate the three magnetic field components across a grid of the Indonesia region, but this grid needs to be within the cap.
Xcoeff = []
Ycoeff = []
Zcoeff = []
count = -1
gridcolat = []
gridlong = []
gm = []

for i in range(Latcentre-theta0,Latcentre+theta0+1):          # runs through latitudes
    for j in range(Longcentre-theta0,Longcentre+theta0+1):    # runs through longitudes
        dist = distance_on_unit_sphere(i,j,Latcentre,Longcentre)*180/math.pi
        # calculates the distance in deg between the grid point and centre of cap
        if dist<=theta0:                                    # eliminates any grid points that are outside the cap and eliminates cap longitude 90 and 270 deg points as these cause singularities
            count+=1
            cl = (90-i)*math.pi/180  # puts latitude into colat in radians
            l = j*math.pi/180        # puts longitude into radians
            gridcolat = gridcolat + [cl]    # makes vectors of the grid colats and longs (original) for later use
            gridlong = gridlong + [l]
            # colat in cap:
            clc = sph_cosine_side(colatradc, cl, longradc-l)
            # longitude in cap
            if j==Longcentre and i>=Latcentre:
                lc = 0
            elif j==Longcentre and i<Latcentre:
                lc = math.pi
            else:
                lc = sph_cosine_angle(colatradc, clc, cl)
            # also need to rotate the X and Y values by angle gamma, such that:
            if j==Longcentre and cl>=colatradc:
                gm = gm + [0]
            elif j==Longcentre and cl<colatradc:
                gm = gm + [math.pi]
            else:
                gm = gm + [sph_cosine_angle(cl, clc, colatradc)]
                
            # need to determine which quadrant the longitude and rotation is in
            if l>longradc:
                lc = 2*math.pi-lc
                gm[count] = 2*math.pi-gm[count]
                
            # now calculate sums for coefficients
            P = assoc_Leg_func(n, A, clc)
            dP = deriv_assoc_Leg_func(n, m, A, clc, P)
            
            # X coefficients are 1/r dU/d(colatcap)
            Xcoeff = Xcoeff + [Bx(m, dP, lc, clc)]
          
            # Y coefficients are -1/rsin(colatcap) dU/d(longcap)
            Ycoeff = Ycoeff + [By(n, m, P, lc, clc)]            
            
            # Z coefficients are du/dr
            Zcoeff = Zcoeff + [Bz(n, m, P, lc, clc)]

<p style='text-align: justify;'>Now to calculate the model field values at each point is a matrix problem. (coeffs) x (gauss) = (field)</p>

In [138]:
Xcalccap = np.dot(np.array(Xcoeff),gauss)
Ycalccap = np.dot(np.array(Ycoeff),gauss)

<p style='text-align: justify;'>Before plotting, need to revert the X and Y values back into geographic coordinates, not cap coords. Calculates the other components as well.</p>

In [139]:
Xcalc = Xcalccap*np.cos(gm)-Ycalccap*np.sin(gm)+data_grid_IGRF.iloc[0:len(Xcoeff), 3]
Ycalc = Xcalccap*np.sin(gm)+Ycalccap*np.cos(gm)+data_grid_IGRF.iloc[0:len(Xcoeff), 4]
Zcalc = np.dot(np.array(Zcoeff),gauss)+data_grid_IGRF.iloc[0:len(Xcoeff), 5]
Dcalc = np.arctan(Ycalc/Xcalc)*180/np.pi
Icalc = np.arctan(Zcalc/np.sqrt(Xcalc**2+Ycalc**2))*180/np.pi
Fcalc = np.sqrt(Xcalc**2+Ycalc**2+Zcalc**2)
Hcalc = np.sqrt(Xcalc**2+Ycalc**2)

<p style='text-align: justify;'>Puts colatitude and longitude back to latitude and longitude in degrees</p>

In [140]:
gridlatd = 90-np.dot(gridcolat,180/np.pi)
gridlongd = np.dot(gridlong,180/np.pi)

<p style='text-align: justify;'>Show G and H gauss coefficients</p>

In [141]:
kk = []
for a in range(mmax + 1):
    b = np.tile(a,(1,a+1))[0].tolist()
    kk = kk + b

gauss_g = np.zeros(len(kk))
gauss_h = np.zeros(len(kk))
q = np.zeros(len(kk)-mmax-1)
g = 0
h = 0
p = -1

for c in range(len(kk)):
    gauss_g[c] = gauss[g]
    if m[c]==0:
        g = g+1
    else:
        g = g+2

for c in range(len(kk)-mmax-1):
    if m[c]==0:
        h = h+3
    else:
        h = h+2
    q[c] = h

for c in range(len(kk)):
    if m[c]==0:
        gauss_h[c] = np.nan
    else:
        p = p+1
        gauss_h[c] = gauss[np.int(q[p])]
        
df_coeff = pd.DataFrame(index=range(len(kk)), columns=['k','m','nkm','kmn','gmk','hmk'])
df_coeff['k'] = kk
df_coeff['m'] = m
df_coeff['nkm'] = ['%.4f' % elem for elem in n]
df_coeff['kmn'] = ['%.4f' % elem for elem in Kmn]
df_coeff['gmk'] = ['%.4f' % elem for elem in gauss_g]
df_coeff['hmk'] = ['%.4f' % elem for elem in gauss_h]

<p style='text-align: justify;'>Saving output file and print the statistics of the model.</p>

In [142]:
# Save the file
fileout = 'tmp\\'+str(year)+'_'+str(mmax)+'_'+str(theta0)+'.txt'
f_out = open(fileout, 'w')

f_out.write('Year    Long      Lat     X         Y        Z          H            D        I       F\n')
for s in range(len(gridlatd)):
    out = '%4.0f %9.2f %9.2f %9.2f %9.2f %9.2f %9.2f %9.2f %9.2f %9.2f\n' %(year,gridlongd[s],gridlatd[s],Xcalc[s],Ycalc[s],Zcalc[s],Hcalc[s],Dcalc[s],Icalc[s],Fcalc[s])
    f_out.write(out)
f_out.close()

print('Statistical results of the SCHA model with kmax = '+str(mmax)+'\n')
print('\t\t\t\t\tX(nT)\tY(nT)\tZ(nT)\nMean Absolute Error (MAE)\t\t%7.2f\t%7.2f\t%7.2f\nRoot Mean Square Difference (RMSD)\t%7.2f\t%7.2f\t%7.2f\n\n\n'%(XMAE,YMAE,ZMAE,XRMSD,YRMSD,ZRMSD))
print('Coefficients of the Spherical Cap Harmonic Analysis of the Indonesian Geomagnetic Data at the Epoch '+str(year)+' for spherical cap = '+str(theta0)+'\u00b0\n')
print('k = cap harmonic degree')
print('m = cap harmonic degree')
print('nkm = non-integer degree of the Legendre function')
print('kmn = Schmidt normalisation constant')
print('gmk, hmk = cap harmonic coefficients\n')

df_coeff.replace('nan', '', regex=True)

Statistical results of the SCHA model with kmax = 7

					X(nT)	Y(nT)	Z(nT)
Mean Absolute Error (MAE)		  99.87	  81.01	 110.25
Root Mean Square Difference (RMSD)	 130.61	 107.61	 156.67



Coefficients of the Spherical Cap Harmonic Analysis of the Indonesian Geomagnetic Data at the Epoch 2015 for spherical cap = 30°

k = cap harmonic degree
m = cap harmonic degree
nkm = non-integer degree of the Legendre function
kmn = Schmidt normalisation constant
gmk, hmk = cap harmonic coefficients



,k,m,nkm,kmn,gmk,hmk
0,0,0,0.0000,1.0000,-484.4968,
1,1,0,4.0837,1.0000,-318.1156,
2,1,1,3.1196,2.5362,-863.6470,436.7040
3,2,0,6.8354,1.0000,292.3059,
4,2,1,6.8354,5.1775,1049.8816,-1087.9306
5,2,2,5.4928,6.1253,312.8332,-775.9311
6,3,0,10.0386,1.0000,-257.2242,
7,3,1,9.7121,7.2160,-1123.3533,1229.1932
8,3,2,9.3733,17.0110,-692.9836,616.9469
9,3,3,7.7524,15.4888,634.6269,-92.5589
